<a href="https://colab.research.google.com/github/componavt/differential_equations/blob/main/src/hill_equation/96_from_picle_to_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





# 🧬📊 Solution Archive for Gene Regulatory ODE System

**Overview:**  
This notebook computes and saves numerical solutions of a two-dimensional gene-regulatory ODE system under a wide range of parameter values. Solutions are saved to disk for later visualization in interactive apps like Streamlit.

---

**Model Equations**  
\begin{cases}
\frac{dx}{dt} = \frac{K\,x^{1/α}}{b^{1/α} + x^{1/α}} \;-\; γ_1\,x,\\[6pt]
\frac{dy}{dt} = \frac{K\,y^{1/α}}{b^{1/α} + y^{1/α}} \;-\; γ_2\,y.
\end{cases}

**Fixed parameters:**  
- b = 1.0, K = 1.0  
- Time: t ∈ [0, 1.0], N = 500 steps  
- Initial condition(s): x₀ = y₀ = b * (1 - ε), ε ≈ 1e-10

**Parameter sweeps:**  
- α ∈ {1e-9, 1e-10, … , 1e-14}  
- γ₁, γ₂ ∈ {0.0, 0.5, 1.0}  
- Solvers: RK45, DOP853, BDF

---

**What this notebook does:**  
1. For each combination of α, γ₁, γ₂, and solver:
   - Integrates the ODE system from a fixed initial condition.
   - Saves time points and trajectories x(t), y(t).
2. Each solution is stored as a dictionary with:
   - Parameters: α, γ₁, γ₂, solver name, initial values
   - Solution arrays: t, x(t), y(t)
   - Status: success or failure
3. All results are saved in a single `pickle` file:  
   📁 `ode_solutions/solutions.pkl`

---

**How to use this in Streamlit:**  
- Load the file with `pickle.load(open(...))`  
- Add controls (checkboxes, sliders) for α, γ₁, γ₂, solver  
- Filter solutions and display interactive plots of x(t), y(t)  

---

*This notebook provides a scalable, solver-agnostic framework for collecting ODE simulation results for downstream visualization and analysis.*

In [1]:
# Modified code for saving ODE solutions for Streamlit as pandas DataFrame
# Cell 1: Parameters and library imports
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
import os

# Model parameters
b = 1.0
t_end = 1.0
K = 1.0
alpha_list = [1e-9 * (10 ** -i) for i in range(6)]  # 1e-9 to 1e-14
print("Alpha values:", alpha_list)

# Time discretization\N = 100  # Reduced number of time steps
t_eval = np.linspace(0, t_end, N)

# Solver methods and gamma parameters
all_methods = ["DOP853", "BDF", "Radau", "RK23"]
gamma_1_list = np.arange(0, 1.01, 0.1)
gamma_2_list = np.arange(0, 1.01, 0.1)

# Initial condition shifts\initial_shifts = [-1e-3, -1e-6, 0.0, 1e-6, 1e-3]
initial_conditions = [(b * (1 + s), b * (1 + s)) for s in initial_shifts]

# Output folder
output_folder = "ode_solutions"
os.makedirs(output_folder, exist_ok=True)

Alpha values: [1e-09, 1.0000000000000002e-10, 1.0000000000000001e-11, 1.0000000000000002e-12, 1.0000000000000002e-13, 1.0000000000000002e-14]


In [2]:
# Cell 2: Define RHS function
def get_rhs(alpha, gamma1, gamma2):
    def rhs(t, xy):
        x = np.clip(xy[0], 1e-20, 1e20)
        y = np.clip(xy[1], 1e-20, 1e20)
        inv_alpha = 1.0 / alpha
        x_alpha = np.exp(np.clip(np.log(x) * inv_alpha, -700, 700))
        y_alpha = np.exp(np.clip(np.log(y) * inv_alpha, -700, 700))
        b_alpha = np.exp(np.clip(np.log(b) * inv_alpha, -700, 700))
        dxdt = K * x_alpha / (b_alpha + x_alpha) - gamma1 * x
        dydt = K * y_alpha / (b_alpha + y_alpha) - gamma2 * y
        return [dxdt, dydt]
    return rhs

In [3]:
# Cell 3: Collect all solutions into list of dicts
records = []
for x0, y0 in initial_conditions:
    for gamma1 in gamma_1_list:
        for gamma2 in gamma_2_list:
            # for each parameter combination, collect solver outputs
            solutions = {}
            for method in all_methods:
                for alpha in alpha_list:
                    rhs = get_rhs(alpha, gamma1, gamma2)
                    try:
                        sol = solve_ivp(rhs, [0, t_end], [x0, y0], t_eval=t_eval, method=method)
                        if sol.success:
                            # store x, y arrays for this method
                            if (x0, y0, gamma1, gamma2, alpha) not in [(r['x0'], r['y0'], r['gamma1'], r['gamma2'], r['alpha']) for r in records]:
                                # first time seeing this parameter set: initialize record
                                records.append({
                                    'x0': x0,
                                    'y0': y0,
                                    'gamma1': gamma1,
                                    'gamma2': gamma2,
                                    'alpha': alpha,
                                    't': sol.t,
                                    'solutions': {method: (sol.y[0], sol.y[1])}
                                })
                            else:
                                # find existing record and add method
                                for rec in records:
                                    if rec['x0']==x0 and rec['y0']==y0 and rec['gamma1']==gamma1 and rec['gamma2']==gamma2 and rec['alpha']==alpha:
                                        rec['solutions'][method] = (sol.y[0], sol.y[1])
                                        break
                    except Exception as e:
                        print(f"Error: method={method}, alpha={alpha}, gamma1={gamma1}, gamma2={gamma2}, x0={x0}: {e}")

# Cell 4: Convert to pandas DataFrame and save
# Each row corresponds to (x0, y0, gamma1, gamma2, alpha), with 't' and 'solutions' columns
df = pd.DataFrame(records)
# Persist DataFrame to pickle
df.to_pickle(os.path.join(output_folder, "solutions.pkl"))
print(f"Saved DataFrame with {len(df)} rows to 'solutions.pkl'")

Saved 2550 unique solution entries to 'solutions.pkl'
Total attempts: 14520
Skipped due to similarity: 11970 (82.44%)
